In [1]:
import mechanicalsoup
from bs4 import BeautifulSoup
import re
import MySQLdb


In [2]:
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://onlinetrading.nse.co.ke/tradeweb111/")

<Response [200]>

In [3]:
file = open('pass.txt', 'r') 
password =str(file.read())

FileNotFoundError: [Errno 2] No such file or directory: 'pass.txt'

In [4]:
browser.select_form(nr=0)

# User credentials
browser['txtLogin'] = 'karis'
browser['txtPassword'] = 'wave1236'

# Login
browser.submit_selected()



<Response [200]>

In [5]:
browser.open('https://onlinetrading.nse.co.ke/tradeweb111/MarketDetails.aspx?wl=Default')
#https://onlinetrading.nse.co.ke/tradeweb111/MarketDetails.aspx?wl=Default
page = browser.get_current_page()

In [6]:
soup = BeautifulSoup(str(page), 'lxml')
script = soup.find_all("script")


In [7]:
jss =  str(script[22])
str_replace_qoutes = jss.replace('\"', '\\"')
str_minus_semi_colon = str_replace_qoutes.replace(";"," ")
str_minus_sdarr = str_minus_semi_colon.replace("sDArr","")
str_minus_equal = str_minus_sdarr.replace("=","")

#everything before sdarr
head, sep, tail = str_minus_equal.partition('(jsiScripsInMW,50) ')
#everything after sdarr
head2, sep2, tail2 = tail.partition('  jsiMWrefreshDelay')
result = re.sub("[\(\[].*?[\)\]]", "", head2)
arr = result.split("' '")
arr[0] = arr[0].replace("\'","")
arr[-1] = arr[-1].replace("\'","")

In [8]:
sdarr = []
sec = []
count =1
for i in range(len(arr)):
    count = count + 1
    sec.append(arr[i])
    if count ==22:
        count = 1
        sdarr.append(sec)
        sec =[]


In [9]:
#print(sdarr)

In [10]:
def sortlist(mylist):
    newlist=[]
    for item in mylist:
        if float(item) > 0:
            newlist.append(item)
    
    if newlist == []:
        newlist = [str(0.00), str(0.00)]
    return newlist

In [12]:
sql = "INSERT INTO market_details (security, last_price, demand_qty, demand_price, supply_price, supply_qty, last_qty, high, low) VALUES "
markets = []
for row in sdarr:
    
    sub_row = [row[2], row[8], row[9], row[10], row[11], row[12]]
    high_low = sortlist(sub_row)
    sql = sql + "( '"+row[0]+"', "+ row[2]+", "+row[5]+", "+row[6]+", "+row[8]+", "+row[7]+", "+row[4]+", "+max(high_low)+", "+min(high_low)+"),"
    markets.append({"security": row[0],
                    "last_price": row[2],
                    "demand_qty": row[5],
                    "demand_price": row[6],
                    "supply_price": row[8],
                    "supply_qty": row[7],
                    "last_qty": row[4],
                    "high": max(high_low),
                    "low":  min(high_low)
                    })
    
sql = sql[:-1] + ";" 
    

print(markets)
# db = MySQLdb.connect(host="localhost",    # your host, usually localhost
#                  user="root",         # your username
#                  passwd="",  # your password
#                  db="nse")        # name of the data base
# 
# db.query(sql)
# db.commit()
# 
# db.close()


[{'security': 'XPRS', 'last_price': '5.95', 'demand_qty': '0', 'demand_price': '0.00', 'supply_price': '0.00', 'supply_qty': '0', 'last_qty': '0', 'high': '5.95', 'low': '5.95'}, {'security': 'KQ', 'last_price': '11.00', 'demand_qty': '0', 'demand_price': '0.00', 'supply_price': '0.00', 'supply_qty': '0', 'last_qty': '100', 'high': '11.55', 'low': '10.20'}, {'security': 'WTK', 'last_price': '139.00', 'demand_qty': '500', 'demand_price': '135.00', 'supply_price': '148.00', 'supply_qty': '1500', 'last_qty': '100', 'high': '148.00', 'low': '139.00'}, {'security': 'UTK', 'last_price': '0.00', 'demand_qty': '0', 'demand_price': '0.00', 'supply_price': '0.00', 'supply_qty': '0', 'last_qty': '0', 'high': '0.0', 'low': '0.0'}, {'security': 'UNGA', 'last_price': '39.25', 'demand_qty': '0', 'demand_price': '0.00', 'supply_price': '0.00', 'supply_qty': '0', 'last_qty': '0', 'high': '39.25', 'low': '39.25'}, {'security': 'UCHM', 'last_price': '1.45', 'demand_qty': '280400', 'demand_price': '1.40',

70
